# ДЗ Гибридные системы

Что делать?
1. Датасет ml-latest
2. Вспомнить подходы, которые мы разбирали
3. Выбрать понравившийся подход к гибридным системам
4. Написать свою


In [1]:
import pandas as pd
import numpy as np

from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise.model_selection import train_test_split
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import KNNWithZScore
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader


from sklearn.model_selection import train_test_split as tts
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from tqdm import tqdm_notebook

In [2]:
movies = pd.read_csv('data/movies/movies.csv')
ratings = pd.read_csv('data/movies/ratings.csv')
tags = pd.read_csv('data/movies/tags.csv')

In [3]:
# сокращенный датасет с фильмами содержащими теги - предполагаем, если пользователь поставил тег, то и оценка более обоснованная
tr = pd.merge(tags, ratings,  how='left', left_on=['movieId','userId'], right_on = ['movieId','userId'])
tr.dropna(inplace=True)

dataset_tr = pd.DataFrame({
    'uid': tr.userId,
    'iid': tr.movieId,
    'rating': tr.rating
})
print(dataset_tr.shape[0])

3476


In [4]:
# полный датасет с рейтингом 
mr = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
mr.dropna(inplace=True)

dataset_mr = pd.DataFrame({
    'uid': mr.userId,
    'iid': mr.movieId,
    'rating': mr.rating
})
print(dataset_mr.shape[0])

100836


#### Построим 2 модели, которые предсказывают оценку фильмам по двум датасетам: для фильмов с тегами и для всех фильмов с жанрами

In [14]:
print('Обучение на сокращенном датасете с тегами:')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset_tr, reader)

kf = KFold(n_splits=3)

algo_tr = KNNBaseline(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False}, verbose=False)


for trainset, testset in kf.split(data):

    algo_tr.fit(trainset)
    predictions_tr = algo_tr.test(testset)
    accuracy.rmse(predictions_tr, verbose=True)

print('Обучение на полном датасете:')    

data = Dataset.load_from_df(dataset_mr, reader)

algo_mr =SVD(n_factors = 50, n_epochs =20, reg_pu = 0.1, reg_qi = 0.1, biased = True)

for trainset, testset in kf.split(data):

    algo_mr.fit(trainset)
    predictions_mr = algo_mr.test(testset)
    accuracy.rmse(predictions_mr, verbose=True)

Обучение на сокращенном датасете с тегами:
RMSE: 0.5434
RMSE: 0.5168
RMSE: 0.5382
Обучение на полном датасете:
RMSE: 0.8702
RMSE: 0.8726
RMSE: 0.8769


#### Еще одна модель с помощью линейной регрессии предсказыват оценку на основе средних по фильмам и пользователям

In [6]:
mean_user = mr.groupby(['userId']).mean().rating
mean_movies = mr.groupby(['movieId']).mean().rating

med_user = mr.groupby(['userId']).median().rating
med_movies = mr.groupby(['movieId']).median().rating

var_user = mr.groupby(['userId']).var().rating
var_movies = mr.groupby(['movieId']).var().rating

movies_st = mr.merge(mean_user, on='userId', suffixes=('', '_meanuser'))
movies_st = movies_st.merge(med_user, on='userId', suffixes=('', '_meduser'))
movies_st = movies_st.merge(var_user, on='userId', suffixes=('', '_varuser'))
movies_st = movies_st.merge(mean_movies, on='movieId', suffixes=('', '_meanmov'))
movies_st = movies_st.merge(med_movies, on='movieId', suffixes=('', '_medmov'))
movies_st = movies_st.merge(var_movies, on='movieId', suffixes=('', '_varmov'))
movies_st['rating_varuser'] = movies_st['rating_varuser'].fillna(0)
movies_st['rating_varmov'] = movies_st['rating_varmov'].fillna(0)

In [7]:
y = movies_st['rating'].astype('int')
X = movies_st

X = X.drop(['rating','timestamp','genres','title'], axis = 1) 
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=800)

print('LinearRegression Lasso L2')

model = linear_model.Lasso(alpha=0.05)
model.fit( X_train, y_train )

y_predict_ln = model.predict(X_test)
score_ln = model.score(X_test, y_test)
mse_ln = mean_squared_error(y_test, y_predict_ln)
sqrt_mse_ln = np.sqrt(mse_ln)


print("MSE: %.5f" % mse_ln)
print("SQ MSE: %.5f" % sqrt_mse_ln)
print("Score: %.5f" % score_ln)
print('R-squared: %.5f' % r2_score(y_test, y_predict_ln))
y_predict_ln = model.predict(X)
print('Предсказанные оценки:', len(y_predict_ln))


LinearRegression Lasso L2
MSE: 0.75225
SQ MSE: 0.86732
Score: 0.37240
R-squared: 0.37240
Предсказанные оценки: 100836


#### Для всех фильмов и пользователей посчитаем эту обобщенную оценку

In [8]:
rez = np.argsort(y_predict_ln)
common_rate = []
for i in rez:
        common_rate.append([X['movieId'][i],  np.round(y_predict_ln[i],2)])

common_rate = pd.DataFrame(common_rate)               
common_rate.columns = ['movieId',  'predict common rating']

common_rate =common_rate.groupby(['movieId']).median()['predict common rating']
pd.DataFrame(common_rate).head(3)

,predict common rating
movieId,
1,3.760
2,3.315
3,3.165


#### Для всех фильмов, которые пользователь еще не смотрел, посчитаем оценку с помощью трех алгоритмов.
#### Сформируем список фильмов, которые  он видел и по которым надо предсказать оценку. И отметим фильм, который он посмотрел последним

In [9]:

user = 140

mr.sort_values('timestamp', inplace=True)

unique_mov = mr['movieId'].unique()
print('Всего фильмов в датасете: ', len(unique_mov))   

user_mov = mr.loc[mr['userId']==user, 'movieId']
print('Пользователь посмотрел: ',len(user_mov))   

movies_to_predict = np.setdiff1d(unique_mov,user_mov)
print('Фильмы для подбора: ',len(movies_to_predict))   

last_user_movie = mr[mr.userId == user].title.unique()[-1]
print('Последний просмотренный фильм: ',last_user_movie)   

Всего фильмов в датасете:  9724
Пользователь посмотрел:  608
Фильмы для подбора:  9116
Последний просмотренный фильм:  Live Free or Die Hard (2007)


In [10]:
rec = []
for iid in movies_to_predict:
    rec.append((iid, user,
                algo_mr.predict(uid=user,iid=iid).est,
                algo_tr.predict(uid=user,iid=iid).est))
    
rec_df = pd.DataFrame(rec, columns=['movieId','userId', 'predictions','predictions_tag'])
rec_df = rec_df.join(movies.set_index('movieId'), on='movieId')
rec_df.sort_values('predictions', ascending=False).head(3)

,movieId,userId,predictions,predictions_tag,title,genres
770,1199,140,4.170687,4.021570,Brazil (1985),Fantasy|Sci-Fi
1513,2329,140,4.157896,4.036985,American History X (1998),Crime|Drama
585,898,140,4.147970,4.048359,"Philadelphia Story, The (1940)",Comedy|Drama|Romance


#### и подберем фильм самый близкий к последнему просмотренному 

In [11]:
movie_vector = {}
num_users = mr.userId.unique().shape[0]

for movie, group in tqdm_notebook(mr.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)
    
    for i in range(len(group.userId.values)):
        u = group.userId.values[i]
        r = group.rating.values[i]
        movie_vector[movie][int(u - 1)] = r

In [12]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard, rogerstanimoto, correlation

titles = []
distances = []

for key in tqdm_notebook(movie_vector.keys()):
    if key == last_user_movie:
        continue
    
    titles.append(key)
    distances.append(correlation(movie_vector[last_user_movie], movie_vector[key]))

best_indexes = np.argsort(distances)
best_movies = [(titles[i], distances[i]) for i in best_indexes]

best_movies_df = pd.DataFrame(best_movies)
best_movies_df.columns = ['title',  'distance from last film']
best_movies_df.sort_values('distance from last film', ascending = False).head(3)

,title,distance from last film
9717,"Postman, The (Postino, Il) (1994)",1.056978
9716,Disclosure (1994),1.053910
9715,Like Water for Chocolate (Como agua para choco...,1.053096


#### Собираем итоговый рейтинг по четырем моделям. 
#### Итоговый вес в рекомендации - это максимальная оценка по жанрам, по тегам, по средним оценкам и самый близкий к последнему просмотренному фильму.

In [13]:
rec_df_dis = pd.merge(rec_df, best_movies_df,   how='left', left_on=['title'], right_on = ['title'])
rec_df_full = pd.merge(rec_df_dis, common_rate,   how='left', left_on=['movieId'], right_on = ['movieId'])
rec_df_full['weight'] = rec_df_full['predictions']+ rec_df_full['predictions_tag']- rec_df_full['distance from last film']+ rec_df_full['predict common rating']
rec_df_full = rec_df_full[['weight','title', 'genres', 'predictions', 'predictions_tag', 'predict common rating','distance from last film']]
rec_df_full.sort_values('weight', ascending = False).head(20)

,weight,title,genres,predictions,predictions_tag,predict common rating,distance from last film
4357,11.881711,Eternal Sunshine of the Spotless Mind (2004),Drama|Romance|Sci-Fi,4.070179,4.638157,3.950,0.776625
1958,11.707930,Fight Club (1999),Action|Crime|Drama|Thriller,4.134653,4.313900,4.065,0.805623
6085,11.570658,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,4.080133,4.165323,4.020,0.694797
6051,11.544758,In Bruges (2008),Comedy|Crime|Drama|Thriller,4.037156,4.298386,3.890,0.680784
744,11.544052,Lesson Faust (1994),Animation|Comedy|Drama|Fantasy,3.455685,4.021570,5.080,1.013203
2866,11.519469,Dr. Goldfoot and the Bikini Machine (1965),Comedy,3.427227,4.021570,5.080,1.009328
8019,11.501719,Into the Forest of Fireflies' Light (2011),Animation|Drama|Fantasy,3.419477,4.021570,5.070,1.009328
452,11.480032,"Thin Line Between Love and Hate, A (1996)",Comedy,3.467790,4.021570,5.000,1.009328
6023,11.472140,There Will Be Blood (2007),Drama|Western,4.024109,4.396915,3.855,0.803884
517,11.452922,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,4.141006,4.187829,4.020,0.895913
